In [1]:
import qiskit
from qiskit import QuantumCircuit, Aer, execute
import numpy as np
import math
from scipy.linalg import lapack
from src.rdmg import rdm_ginibre
import sys
from qiskit import QuantumRegister,ClassicalRegister,QuantumCircuit


In [3]:
import numpy as np

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import BasicAer
from qiskit.compiler import transpile
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.quantum_info import process_fidelity
from qiskit.quantum_info import state_fidelity
from qiskit import execute, Aer
from qiskit.quantum_info import DensityMatrix
import matplotlib.pyplot as plt

from qiskit.extensions import RXGate, XGate, CXGate



In [2]:
def Pauli(j):
    if j == 0:
        return Matrix([[1,0],[0,1]])
    elif j == 1:
        return Matrix([[0,1],[1,0]])
    elif j == 2:
        return Matrix([[0,-1j],[1j,0]])
    elif j == 3:
        return Matrix([[1,0],[0,-1]])

In [4]:
from qiskit.providers.aer import StatevectorSimulator
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter

C:\Users\dougl\AppData\Local\Temp\ipykernel_14748\3424116430.py:2: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter


In [5]:
qiskit.IBMQ.save_account('MY_API_TOKEN', overwrite=True)

C:\Users\dougl\AppData\Local\Temp\ipykernel_14748\1308432029.py:1: DeprecationWarning: The package qiskit.providers.ibmq is being deprecated. Please see https://ibm.biz/provider_migration_guide to get instructions on how to migrate to qiskit-ibm-provider (https://github.com/Qiskit/qiskit-ibm-provider) and qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  qiskit.IBMQ.save_account('2537d4125e263262381a269464d18da5d1d1a37a4a00534ce31274e9e2186327f9f93c3cf4265d538d599408669cf7ebe06b299ead2876a3035550c0071af3b6', overwrite=True)
C:\Users\dougl\AppData\Local\Temp\ipykernel_14748\1308432029.py:1: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_

In [7]:
from qiskit import *
#nshots = 8192
nshots =  10000
IBMQ.load_account()
provider = qiskit.IBMQ.get_provider(hub='ibm-q', group='open', project='main')
#provider = qiskit.IBMQ.get_provider(hub = 'ibm-q-research-2', group = 'federal-uni-sant-1', project = 'main')
device = provider.get_backend('ibm_osaka')
#device = provider.get_backend('ibmq_quito')
#device = provider.get_backend('ibm_nairobi')
simulator = Aer.get_backend('qasm_simulator')
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor

ibmqfactory.load_account:WARNING:2024-01-19 14:31:35,373: Credentials are already in use. The existing account in the session will be replaced.


In [8]:
from qiskit.tools.monitor import backend_overview, backend_monitor
backend_overview()

ibm_osaka                    ibm_kyoto                    ibm_brisbane
---------                    ---------                    ------------
Num. Qubits:  127            Num. Qubits:  127            Num. Qubits:  127
Pending Jobs: 13             Pending Jobs: 0              Pending Jobs: 46
Least busy:   False          Least busy:   True           Least busy:   False
Operational:  True           Operational:  True           Operational:  True
Avg. T1:      281.3          Avg. T1:      217.5          Avg. T1:      216.7
Avg. T2:      159.6          Avg. T2:      131.7          Avg. T2:      149.7





In [ ]:
#def filtro(valor):
#    if abs(valor) < 0.000000001:
#        return 0
#    else:
#       return valor

def filtro(valor):
    filtered_valor = np.where(np.logical_and(valor < 0, np.isclose(valor, 0, atol=1e-5)), 0, valor)
    return filtered_valor    

def get_abs_lambda(rho):

    n = np.shape(rho)[0]
    w, u, _ = lapack.zheev(rho, overwrite_a=True)
    psi = 0
    for i in range(len(u)):
        u[:, i] = u[:, i]/np.linalg.norm(u[:, i])
    for i in range(n):
        #rho += w[i] * np.outer(u[:, i], u[:, i].conj()) # se precisar do rho
        w[i] = filtro(w[i])
        #print(f'w[{i}] = ', w[i])
        if w[i] < 0.0000001:
            w[i] == 0
        psi += math.sqrt(w[i])*np.eye(n)[:, i]
    return psi, u

In [ ]:
def get_rho_circuit(psi, rho, u):
    n_qubits = int(np.ceil(np.log2(np.shape(rho)[0])))
    d = int(2*n_qubits)
    #print(n_qubits)

    qr = QuantumRegister(d)
    #cr = ClassicalRegister(n_qubits)
    if n_qubits == 1:
        lista = [qr[0]]
    else:
        lista = []
        for i in range(n_qubits):
            #lista = [i for i in qr]#[:-1]#[,qr[1]]
            lista.append(qr[i])
    U_op = Operator(u)
    qc = QuantumCircuit(qr, name='qc_initialize')

    qc.initialize(psi, lista)
    for i in range(int(n_qubits)):
        qc.cx(i, int(n_qubits)+i)
    qc.append(U_op,   list(range(n_qubits)))
            
    return qc

In [ ]:
def Psi00(th):
    return np.array([[np.cos(th)],[0],[0],[np.sin(th)]]);

def Psi01(ph):
    return np.array([[0],[np.sin(ph)],[np.cos(ph)],[0]]);

def Psi11(ph):
    return np.array([[0],[np.cos(ph)],[-np.sin(ph)],[0]]);

def Psi10(th):
    return np.array([[-np.sin(th)],[0],[0],[np.cos(th)]]);

In [ ]:
th=np.pi/4
ph=np.pi/4

def rhoX(th,ph,p00,p01,p10,p11):
    return p00*(Psi00(th)*Psi00(th).T) +  p01*(Psi01(ph)*Psi01(ph).T) + p10*(Psi10(th)*Psi10(th).T) + p11*(Psi11(ph)*Psi11(ph).T)

rhoX(th,ph,1/2,1/2,0,0)

In [ ]:
###Teste implementação/simulação estado tipo X escolhido 

rhoX_ = np.array(rhoX(np.pi/4,np.pi/4,1/2,0,0,1/2))
n_qubits = int(np.ceil(np.log2(np.shape(rhoX_)[0])))
d = int(2*n_qubits)
qr = QuantumRegister(d)
qc = QuantumCircuit(qr)
psi, u = get_abs_lambda(rhoX_)
qc_ = get_rho_circuit(psi, rhoX_, u)
qc.append(qc_,[qr[0],qr[1],qr[2],qr[3]])

qstc = state_tomography_circuits(qc, [qr[0],qr[1]]); # [qr[0],qr[1]]circuito para TEQ
job_sim = qiskit.execute(qstc, Aer.get_backend('qasm_simulator'), shots = nshots)
print(job_sim.job_id()); job_monitor(job_sim)
qstf = StateTomographyFitter(job_sim.result(), qstc) # ajusta os dados
rhofit = qstf.fit(method='lstsq'); #extrai o operador densidade
fid = state_fidelity(rhoX_, rhofit); #fidelidade entre estado tomografado e rhogerado aleatóriamente
print(rhofit)
print(fid)

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, Aer, execute
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity

# Definição das funções get_rho_circuit, get_abs_lambda e rhoX
# (certifique-se de ter essas funções definidas no seu código)

# Parâmetros
rhoX_ = np.array(rhoX(np.pi/4, np.pi/4, 1/2, 0, 0, 1/2))
n_qubits = int(np.ceil(np.log2(np.shape(rhoX_)[0])))
d = int(2 * n_qubits)
qr = QuantumRegister(d)
qc = QuantumCircuit(qr)

# Implementação do estado X
psi, u = get_abs_lambda(rhoX_)

# Criação do circuito de implementação do estado X
qc_impl = get_rho_circuit(psi, rhoX_, u)

# Composição dos circuitos em qc
qc += qc_impl

# Realiza a tomografia de estado
qstc = state_tomography_circuits(qc, [qr[0], qr[1]])

# Executa a simulação
nshots = 1024  # Defina o número adequado de shots
job_sim = execute(qstc, Aer.get_backend('qasm_simulator'), shots=nshots)
print(job_sim.job_id())
# job_monitor(job_sim)  # Opcional - pode ser necessário importar a função job_monitor do módulo qiskit.tools.monitor

# Realiza o ajuste dos dados
qstf = StateTomographyFitter(job_sim.result(), qstc)
rhofit = qstf.fit(method='lstsq')

# Calcula a fidelidade entre o estado tomografado e o estado original
fid = state_fidelity(rhoX_, rhofit)

print(rhofit)
print(fid)

In [ ]:
###Teste implementação/simulação estado tipo X escolhido 

rhoX_ = np.array(rhoX(np.pi/4,np.pi/4,1/2,0,0,1/2))
n_qubits = int(np.ceil(np.log2(np.shape(rhoX_)[0])))
d = int(2*n_qubits)
qr = QuantumRegister(d)
qc = QuantumCircuit(qr)
psi, u = get_abs_lambda(rhoX_)
qc_ = get_rho_circuit(psi, rhoX_, u)
qc.append(qc_,[qr[0],qr[1],qr[2],qr[3]])

qstc = state_tomography_circuits(qc, [qr[0],qr[1]]); # [qr[0],qr[1]]circuito para TEQ
job_sim = qiskit.execute(qstc, Aer.get_backend('qasm_simulator'), shots = nshots)
print(job_sim.job_id()); job_monitor(job_sim)
qstf = StateTomographyFitter(job_sim.result(), qstc) # ajusta os dados
rhofit = qstf.fit(method='lstsq'); #extrai o operador densidade
fid = state_fidelity(rhoX_, rhofit); #fidelidade entre estado tomografado e rhogerado aleatóriamente
print(rhofit)
print(fid)

In [ ]:
def pTraceL_num(dl, dr, rhoLR):
    # Returns the left partial trace over the 'left' subsystem of rhoLR
    rhoR = np.zeros((dr, dr), dtype=complex)
    for j in range(0, dr):
        for k in range(j, dr):
            for l in range(0, dl):
                rhoR[j,k] += rhoLR[l*dr+j,l*dr+k]
            if j != k:
                rhoR[k,j] = np.conj(rhoR[j,k])
    return rhoR


def pTraceR_num(dl, dr, rhoLR):
    # Returns the right partial trace over the 'right' subsystem of rhoLR
    rhoL = np.zeros((dl, dl), dtype=complex)
    for j in range(0, dl):
        for k in range(j, dl):
            for l in range(0, dr):
                rhoL[j,k] += rhoLR[j*dr+l,k*dr+l]
        if j != k:
            rhoL[k,j] = np.conj(rhoL[j,k])
    return rhoL

In [ ]:
###Teste implementação/simulação estado tipo X escolhido 
import numpy as np
from numpy import zeros, conjugate, sqrt
def eVals(A):
    d = A.shape[0]; evals = zeros(d,1)
    eig = A.eigenvects()
    ne = 0
    j = 0
    lk = 0
    while ne < d:
        mult = eig[j][1]
        ne += mult
        nk = lk + mult
        for k in range(lk,nk):
            evals[k] = eig[j][0]
        lk = nk
        j += 1
    return evals

def tp_(x,y):
    return np.kron(x,y)

def coh_l1(rho):
        d = rho.shape[0]
        C = 0
        for j in range(0, d-1):
            for k in range(j+1, d):
                C += abs(rho[j, k])
        return 2 * C

def concurrence(rho):
    pauli = np.array([[0, 1], [1, 0]])
    tp_pauli = np.kron(pauli, pauli)
    rho_conj = np.conjugate(rho)
    
    R = rho.dot(tp_pauli).dot(rho_conj).dot(tp_pauli)
    ev = np.linalg.eigvals(R)
    evm = np.max(np.abs(ev))
    
    C = 2.0 * np.sqrt(np.abs(evm)) - np.sqrt(np.abs(ev[0])) - np.sqrt(np.abs(ev[1])) - np.sqrt(np.abs(ev[2])) - np.sqrt(np.abs(ev[3]))
    C = np.maximum(C, 0.0)
    
    return C

def varredura():
    space = np.linspace(0,np.pi,10)
    conc = []
    coh = []
    for i in space:
        rhoX_ = np.array(rhoX(np.pi/4,i,1/2,0,0,1/2))
        n_qubits = int(np.ceil(np.log2(np.shape(rhoX_)[0])))
        d = int(2*n_qubits)
        qr = QuantumRegister(d)
        qc = QuantumCircuit(qr)
        psi, u = get_abs_lambda(rhoX_)
        qc_ = get_rho_circuit(psi, rhoX_, u)
        qc.append(qc_,[qr[0],qr[1],qr[2],qr[3]])

        qstc = state_tomography_circuits(qc, [qr[0],qr[1]]); # [qr[0],qr[1]]circuito para TEQ
        job_sim = qiskit.execute(qstc, Aer.get_backend('qasm_simulator'), shots = nshots)
        print(job_sim.job_id()); job_monitor(job_sim)
        qstf = StateTomographyFitter(job_sim.result(), qstc) # ajusta os dados
        rhofit = qstf.fit(method='lstsq'); #extrai o operador densidade
        conc.append(concurrence(rhofit))
        coh.append(coh_l1(rhofit))
        fid = state_fidelity(rhoX_, rhofit); #fidelidade entre estado tomografado e rhogerado aleatóriamente
        #print(rhofit)
        #print(fid)
    plt.plot(space, conc)
    plt.plot(space, coh)
    plt.show()
varredura()

In [ ]:
###Teste implementação/simulação estado tipo X escolhido 
import numpy as np
from numpy import zeros, conjugate, sqrt
def eVals(A):
    d = A.shape[0]; evals = zeros(d,1)
    eig = A.eigenvects()
    ne = 0
    j = 0
    lk = 0
    while ne < d:
        mult = eig[j][1]
        ne += mult
        nk = lk + mult
        for k in range(lk,nk):
            evals[k] = eig[j][0]
        lk = nk
        j += 1
    return evals

def tp_(x,y):
    return np.kron(x,y)

def coh_l1(rho):
        d = rho.shape[0]
        C = 0
        for j in range(0, d-1):
            for k in range(j+1, d):
                C += abs(rho[j, k])
        return 2 * C

def concurrence(rho):
    pauli = np.array([[0, 1], [1, 0]])
    tp_pauli = np.kron(pauli, pauli)
    rho_conj = np.conjugate(rho)
    
    R = rho.dot(tp_pauli).dot(rho_conj).dot(tp_pauli)
    ev = np.linalg.eigvals(R)
    evm = np.max(np.abs(ev))
    
    C = 2.0 * np.sqrt(np.abs(evm)) - np.sqrt(np.abs(ev[0])) - np.sqrt(np.abs(ev[1])) - np.sqrt(np.abs(ev[2])) - np.sqrt(np.abs(ev[3]))
    C = np.maximum(C, 0.0)
    
    return C

def varredura():
    space = np.linspace(0,np.pi,10)
    conc = []
    coh = []
    for i in space:
        rhoX_ = np.array(rhoX(np.pi/4,i,1/2,0,0,1/2))
        n_qubits = int(np.ceil(np.log2(np.shape(rhoX_)[0])))
        d = int(2*n_qubits)
        qr = QuantumRegister(d)
        qc = QuantumCircuit(qr)
        psi, u = get_abs_lambda(rhoX_)
        qc_ = get_rho_circuit(psi, rhoX_, u)
        qc.append(qc_,[qr[0],qr[1],qr[2],qr[3]])

        qstc = state_tomography_circuits(qc, [qr[0],qr[1]]); # [qr[0],qr[1]]circuito para TEQ
        job_sim = qiskit.execute(qstc, Aer.get_backend('qasm_simulator'), shots = nshots)
        print(job_sim.job_id()); job_monitor(job_sim)
        qstf = StateTomographyFitter(job_sim.result(), qstc) # ajusta os dados
        rhofit = qstf.fit(method='lstsq'); #extrai o operador densidade
        conc.append(concurrence(rhofit))
        coh.append(coh_l1(rhofit))
        fid = state_fidelity(rhoX_, rhofit); #fidelidade entre estado tomografado e rhogerado aleatóriamente
        #print(rhofit)
        #print(fid)
    plt.plot(space, conc)
    plt.plot(space, coh)
    plt.show()
varredura()

In [ ]:
def varredura2():
    space = np.linspace(0,1,10)
    conc = []
    coh = []
    for i in space:
        rhoX_ = np.array(rhoX(np.pi/4,np.pi/4,i,0,0,1-i))
        n_qubits = int(np.ceil(np.log2(np.shape(rhoX_)[0])))
        d = int(2*n_qubits)
        qr = QuantumRegister(d)
        qc = QuantumCircuit(qr)
        psi, u = get_abs_lambda(rhoX_)
        qc_ = get_rho_circuit(psi, rhoX_, u)
        qc.append(qc_,[qr[0],qr[1],qr[2],qr[3]])

        qstc = state_tomography_circuits(qc, [qr[0],qr[1]]); # [qr[0],qr[1]]circuito para TEQ
        job_sim = qiskit.execute(qstc, Aer.get_backend('qasm_simulator'), shots = nshots)
        print(job_sim.job_id()); job_monitor(job_sim)
        qstf = StateTomographyFitter(job_sim.result(), qstc) # ajusta os dados
        rhofit = qstf.fit(method='lstsq'); #extrai o operador densidade
        conc.append(concurrence(rhofit))
        coh.append(coh_l1(rhofit))
        fid = state_fidelity(rhoX_, rhofit); #fidelidade entre estado tomografado e rhogerado aleatóriamente
        #print(rhofit)
        #print(fid)
    plt.plot(space, conc)
    plt.plot(space, coh)
    plt.show()
varredura2()

## Ajustes nos gráficos 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import qiskit
from qiskit import QuantumRegister, QuantumCircuit, Aer
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity

def calculate_data():
    nshots = 8192  # You can adjust the number of shots
    space = np.linspace(0, 1, 10)
    conc = []
    coh = []

    for i in space:
        rhoX_ = np.array(rhoX(np.pi/4, np.pi/4, i, 0, 0, 1-i))
        n_qubits = int(np.ceil(np.log2(np.shape(rhoX_)[0])))
        d = int(2 * n_qubits)
        qr = QuantumRegister(d)
        qc = QuantumCircuit(qr)
        psi, u = get_abs_lambda(rhoX_)
        qc_ = get_rho_circuit(psi, rhoX_, u)
        qc.append(qc_, [qr[0], qr[1], qr[2], qr[3]])

        qstc = state_tomography_circuits(qc, [qr[0], qr[1]])  # [qr[0], qr[1]] circuit for TEQ
        job_sim = qiskit.execute(qstc, Aer.get_backend('qasm_simulator'), shots=nshots)
        print(job_sim.job_id())
        job_monitor(job_sim)
        qstf = StateTomographyFitter(job_sim.result(), qstc)  # Adjusts the data
        rhofit = qstf.fit(method='lstsq')  # Extract the density operator
        conc.append(concurrence(rhofit))
        coh.append(coh_l1(rhofit))
        fid = state_fidelity(rhoX_, rhofit)  # Fidelity between tomographed state and randomly generated state
        # print(rhofit)
        # print(fid)

    return space, conc, coh

In [ ]:
# Call the function to calculate the data
space, conc, coh = calculate_data()

In [ ]:
def generate_graphs(space, conc, coh):
    plt.plot(space, conc, label=r'$E_{C}^{sim}$')
    plt.plot(space, coh, label=r'$C_{l_{1}}^{sim}$')
    plt.xlabel('Parameter Space')
    plt.ylabel('Values')
    plt.legend()
    plt.show()

In [ ]:
# Call the function to generate the graphs
generate_graphs(space, conc, coh)

In [ ]:
# test_state 
rhoX(th,ph,p00,p01,p10,p11)

In [ ]:
t = np.linspace(0,1,100)
y = [concurrence(rhoX(np.pi/4, np.pi/4, i, 0, 0, 1-i)) for i in t]
z = [coh_l1(rhoX(np.pi/4, np.pi/4, i, 0, 0, 1-i)) for i in t]
plt.plot(t,y)
plt.plot(t,z)
plt.show()

In [ ]:
# Dados para o gráfico teórico
t = np.linspace(0, 1, 100)
y_teoria = [concurrence(rhoX(np.pi/4, np.pi/4, i, 0, 0, 1-i)) for i in t]
z_teoria = [coh_l1(rhoX(np.pi/4, np.pi/4, i, 0, 0, 1-i)) for i in t]

# Plot do gráfico teórico
plt.plot(t, y_teoria, label='Teórico Concurrence')
plt.plot(t, z_teoria, label='Teórico Coherence')

    # Retornar também os valores teóricos de conc e coh
    return space, conc, coh, y_teoria, z_teoria

In [ ]:

def calculate_data():
    nshots = 8192  # You can adjust the number of shots
    space = np.linspace(0, 1, 10)
    conc = []
    coh = []

    for i in space:
        rhoX_ = np.array(rhoX(np.pi/4, np.pi/4, i, 0, 0, 1-i))
        n_qubits = int(np.ceil(np.log2(np.shape(rhoX_)[0])))
        d = int(2 * n_qubits)
        qr = QuantumRegister(d)
        qc = QuantumCircuit(qr)
        psi, u = get_abs_lambda(rhoX_)
        qc_ = get_rho_circuit(psi, rhoX_, u)
        qc.append(qc_, [qr[0], qr[1], qr[2], qr[3]])

        qstc = state_tomography_circuits(qc, [qr[0], qr[1]])  # [qr[0], qr[1]] circuit for TEQ
        job_sim = qiskit.execute(qstc, Aer.get_backend('qasm_simulator'), shots=nshots)
        print(job_sim.job_id())
        job_monitor(job_sim)
        qstf = StateTomographyFitter(job_sim.result(), qstc)  # Adjusts the data
        rhofit = qstf.fit(method='lstsq')  # Extract the density operator
        conc.append(concurrence(rhofit))
        coh.append(coh_l1(rhofit))
        fid = state_fidelity(rhoX_, rhofit)  # Fidelity between tomographed state and randomly generated state
        # print(rhofit)
        # print(fid)    
    
    # Dados para o gráfico teórico
        t = np.linspace(0, 1, 100)
        y_teoria = [concurrence(rhoX(np.pi/4, np.pi/4, i, 0, 0, 1-i)) for i in t]
        z_teoria = [coh_l1(rhoX(np.pi/4, np.pi/4, i, 0, 0, 1-i)) for i in t]

    # Retornar também os valores teóricos de conc e coh
    return space, conc, coh, y_teoria, z_teoria
   
# Call the function to calculate the data
space, conc, coh, y_teoria, z_teoria = calculate_data()

In [ ]:
# Função para gerar os gráficos
def generate_graphs(space, conc, coh, y_teoria, z_teoria):
    plt.plot(space, conc, 'x', label=r'$E_{C}^{sim}$')
    plt.plot(space, coh, 'o', label=r'$C_{l_{1}}^{sim}$')
    plt.plot(t, y_teoria, label=r'$E_{C}^{the}$')
    plt.plot(t, z_teoria, label=r'$C_{l_{1}}^{the}$')
    plt.xlabel('Parameter Space')
    plt.ylabel('Values')
    plt.legend()
    plt.show()

In [ ]:
# Call the function to generate the graphs
generate_graphs(space, conc, coh, y_teoria, z_teoria)   



### Gráficos dos resultados Teóricos, Simulados e demonstrações experimentais na IBMQ

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import qiskit
from qiskit import QuantumRegister, QuantumCircuit, Aer
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity

nshots = 1000000 # You can adjust the number of shots

# Função para calcular os dados
def q_X_STATE():
    space = np.linspace(0, 1, 11)
    conc = []
    coh = []

    for i in space:
        rhoX_ = np.array(rhoX(np.pi/8,np.pi/8,i/3,(1-i)/3,2*i/3,2*(1-i)/3))
        n_qubits = int(np.ceil(np.log2(np.shape(rhoX_)[0])))
        d = int(2*n_qubits)
        qr = QuantumRegister(d)
        qc = QuantumCircuit(qr)
        psi, u = get_abs_lambda(rhoX_)
        qc_ = get_rho_circuit(psi, rhoX_, u)
        qc.append(qc_,[qr[0],qr[1],qr[2],qr[3]])

        qstc = state_tomography_circuits(qc, [qr[0],qr[1]]); # [qr[0],qr[1]]circuito para TEQ
        job_sim = qiskit.execute(qstc, Aer.get_backend('qasm_simulator'), shots = nshots)
        #print(rhoX_[1])
        #print(job_sim.job_id()); job_monitor(job_sim)
        qstf = StateTomographyFitter(job_sim.result(), qstc) # ajusta os dados
        rhofit = qstf.fit(method='lstsq'); #extrai o operador densidade

        conc.append(concurrence(rhofit))
        coh.append(coh_l1(rhofit))
        fid = state_fidelity(rhoX_, rhofit)
        # Dados para o gráfico teórico
        t = np.linspace(0, 1, 100)
        Ec_teor = [concurrence(rhoX(np.pi/8,np.pi/8,i/3,(1-i)/3,2*i/3,2*(1-i)/3)) for i in t]
        Coh_teor = [coh_l1(rhoX(np.pi/8,np.pi/8,i/3,(1-i)/3,2*i/3,2*(1-i)/3)) for i in t]

    return space, conc, coh, Ec_teor, Coh_teor 

# Chamar a função para calcular os dados
space, conc, coh, Ec_teor, Coh_teor = q_X_STATE()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import qiskit
from qiskit import QuantumRegister, QuantumCircuit, Aer
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity

nshots = 1000000 # You can adjust the number of shots

# Função para calcular os dados
def q_X_STATE():
    space = np.linspace(0, 1, 11)
    conc = []
    coh = []

    for i in space:
        rhoX_ = np.array(rhoX(np.pi/8,np.pi/8,i/3,(1-i)/3,2*i/3,2*(1-i)/3))
        n_qubits = int(np.ceil(np.log2(np.shape(rhoX_)[0])))
        d = int(2*n_qubits)
        qr = QuantumRegister(d)
        qc = QuantumCircuit(qr)
        psi, u = get_abs_lambda(rhoX_)
        qc_ = get_rho_circuit(psi, rhoX_, u)
        qc.append(qc_,[qr[0],qr[1],qr[2],qr[3]])

        qstc = state_tomography_circuits(qc, [qr[0],qr[1]]); # [qr[0],qr[1]]circuito para TEQ
        job_sim = qiskit.execute(qstc, Aer.get_backend('qasm_simulator'), shots = nshots)
        #print(rhoX_[1])
        #print(job_sim.job_id()); job_monitor(job_sim)
        qstf = StateTomographyFitter(job_sim.result(), qstc) # ajusta os dados
        rhofit = qstf.fit(method='lstsq'); #extrai o operador densidade

        conc.append(concurrence(rhofit))
        coh.append(coh_l1(rhofit))
        fid = state_fidelity(rhoX_, rhofit)
        # Dados para o gráfico teórico
        t = np.linspace(0, 1, 100)
        Ec_teor = [concurrence(rhoX(np.pi/8,np.pi/8,i/3,(1-i)/3,2*i/3,2*(1-i)/3)) for i in t]
        Coh_teor = [coh_l1(rhoX(np.pi/8,np.pi/8,i/3,(1-i)/3,2*i/3,2*(1-i)/3)) for i in t]

    return space, conc, coh, Ec_teor, Coh_teor 

# Chamar a função para calcular os dados
space, conc, coh, Ec_teor, Coh_teor = q_X_STATE()

In [ ]:
# Função para gerar os gráficos
def generate_graphs(space, conc, coh, Ec_teor, Coh_teor):
    plt.plot(space, conc, 'x', label=r'$E_{C}^{sim}$')
    plt.plot(space, coh, '.', label=r'$C_{l_{1}}^{sim}$')
    plt.plot(t, Ec_teor, label=r'$E_{C}^{the}$')
    plt.plot(t, Coh_teor, label=r'$C_{l_{1}}^{the}$')
    plt.xlabel(r'$p_{00}$')
    plt.ylabel('Quantumness')
    plt.legend()
    plt.show()


In [ ]:
# Chamar a função para gerar os gráficos
generate_graphs(space, conc, coh, Ec_teor, Coh_teor)

## Adaptado com salvamento de dados

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import qiskit
from qiskit import QuantumRegister, QuantumCircuit, Aer
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity

nshots = 10000 # You can adjust the number of shots
# Função para calcular os dados


import pickle
def q_X_STATE():
    space = np.linspace(0, 1, 11)
    conc = []
    coh = []
    for i in space:
        rhoX_ = np.array(rhoX(np.pi/8,np.pi/8,i/3,(1-i)/3,2*i/3,2*(1-i)/3))
        n_qubits = int(np.ceil(np.log2(np.shape(rhoX_)[0])))
        d = int(2*n_qubits)
        qr = QuantumRegister(d)
        qc = QuantumCircuit(qr)
        psi, u = get_abs_lambda(rhoX_)
        qc_ = get_rho_circuit(psi, rhoX_, u)
        qc.append(qc_,[qr[0],qr[1],qr[2],qr[3]])
        qstc = state_tomography_circuits(qc, [qr[0],qr[1]]); # [qr[0],qr[1]]circuito para TEQ
        job_sim = qiskit.execute(qstc, Aer.get_backend('qasm_simulator'), shots = nshots)
        #print(rhoX_[1])
        #print(job_sim.job_id()); job_monitor(job_sim)
        qstf = StateTomographyFitter(job_sim.result(), qstc) # ajusta os dados
        rhofit = qstf.fit(method='lstsq'); #extrai o operador densidade
        conc.append(concurrence(rhofit))
        coh.append(coh_l1(rhofit))
        fid = state_fidelity(rhoX_, rhofit)
        
        # Dados para o gráfico teórico
        t = np.linspace(0, 1, 100)
        Ec_teor = [concurrence(rhoX(np.pi/8,np.pi/8,i/3,(1-i)/3,2*i/3,2*(1-i)/3)) for i in t]
        Coh_teor = [coh_l1(rhoX(np.pi/8,np.pi/8,i/3,(1-i)/3,2*i/3,2*(1-i)/3)) for i in t]
        
    ''' Escreve'''
    nome = 'coh_sim'
    with open(f'{nome}.pkl', 'wb') as f:
        pickle.dump(coh, f)
    nome = 'conc_sim'
    with open(f'{nome}.pkl', 'wb') as f:
        pickle.dump(conc, f)
  
    return space, Ec_teor, Coh_teor

# Chamar a função para calcular os dados
space, Ec_teor, Coh_teor = q_X_STATE()

In [ ]:
  ''' Le'''
path = 'coh_sim.pkl'
with open(path, 'rb') as f:
    coh_sim = pickle.load(f) #
path = 'conc_sim.pkl'
with open(path, 'rb') as f:
    conc_sim = pickle.load(f) # é o que é lido

In [ ]:
# Dados para o gráfico teórico
t = np.linspace(0, 1, 100)
Ec_teor = [concurrence(rhoX(np.pi/8,np.pi/8,i/3,(1-i)/3,2*i/3,2*(1-i)/3)) for i in t]
Coh_teor = [coh_l1(rhoX(np.pi/8,np.pi/8,i/3,(1-i)/3,2*i/3,2*(1-i)/3)) for i in t]

In [ ]:
# Função para gerar os gráficos
def generate_graphs(space, conc, coh, Ec_teor, Coh_teor):
    plt.plot(space, conc, 'x', label=r'$E_{C}^{sim}$')
    plt.plot(space, coh, '.', label=r'$C_{l_{1}}^{sim}$')
    plt.plot(t, Ec_teor, label=r'$E_{C}^{the}$')
    plt.plot(t, Coh_teor, label=r'$C_{l_{1}}^{the}$')
    plt.xlabel(r'$p_{00}$')
    plt.ylabel('Quantumness')
    plt.legend()
    plt.show()

In [ ]:
# Chamar a função para gerar os gráficos
generate_graphs(space, conc_sim, coh_sim, Ec_teor, Coh_teor)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import qiskit
from qiskit import QuantumRegister, QuantumCircuit, Aer
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity

nshots = 10000 # You can adjust the number of shots
# Função para calcular os dados


import pickle
def q_X_state_exp():
    space = np.linspace(0, 1, 11)
    conc_exp = []
    coh_exp = []
    for i in space:
        rhoX_ = np.array(rhoX(np.pi/8,np.pi/8,i/3,(1-i)/3,2*i/3,2*(1-i)/3))
        n_qubits = int(np.ceil(np.log2(np.shape(rhoX_)[0])))
        d = int(2*n_qubits)
        qr = QuantumRegister(d)
        qc = QuantumCircuit(qr)
        psi, u = get_abs_lambda(rhoX_)
        qc_ = get_rho_circuit(psi, rhoX_, u)
        qc.append(qc_,[qr[0],qr[1],qr[2],qr[3]])
        qstc = state_tomography_circuits(qc, [qr[0],qr[1]]); # [qr[0],qr[1]]circuito para TEQ
        #job_sim = qiskit.execute(qstc, Aer.get_backend('qasm_simulator'), shots = nshots)
        job_exp = qiskit.execute(qstc, backend = device, shots = nshots)
        print(job_exp.job_id()); job_monitor(job_exp)
        
        qstf = StateTomographyFitter(job_exp.result(), qstc) # ajusta os dados
        rhofit_exp = qstf.fit(method='lstsq'); #extrai o operador densidade
        

        conc_exp.append(concurrence(rhofit_exp))
        coh_exp.append(coh_l1(rhofit_exp))
        fid_ = state_fidelity(rhoX_, rhofit_exp)
        
        # Dados para o gráfico teórico
        t = np.linspace(0, 1, 100)
        Ec_teor = [concurrence(rhoX(np.pi/8,np.pi/8,i/3,(1-i)/3,2*i/3,2*(1-i)/3)) for i in t]
        Coh_teor = [coh_l1(rhoX(np.pi/8,np.pi/8,i/3,(1-i)/3,2*i/3,2*(1-i)/3)) for i in t]
        
    ''' Escreve'''
    nome = 'coh_expt'
    with open(f'{nome}.pkl', 'wb') as f:
        pickle.dump(coh_exp, f)
    nome = 'conc_expt'
    with open(f'{nome}.pkl', 'wb') as f:
        pickle.dump(conc_exp, f)
  
    return space, Ec_teor, Coh_teor

# Chamar a função para calcular os dados
space, Ec_teor, Coh_teor = q_X_state_exp()





In [ ]:
  ''' Le'''
path = 'coh_sim.pkl'
with open(path, 'rb') as f:
    coh_sim = pickle.load(f) #
path = 'conc_sim.pkl'
with open(path, 'rb') as f:
    conc_sim = pickle.load(f)
    
path = 'coh_expt.pkl'
with open(path, 'rb') as f:
    coh_exp = pickle.load(f) #    
    
path = 'conc_expt.pkl'
with open(path, 'rb') as f:
    conc_exp = pickle.load(f)    # é o que é lido

In [ ]:

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size':11}); plt.figure(figsize = (7,5), dpi = 100)

def generate_graphs(space, conc, coh, conc_exp, coh_exp, Ec_teor, Coh_teor):
    
    plt.plot(space, conc, 'x', label=r'$E_{C}^{sim}$')
    plt.plot(space, coh, 'o', label=r'$C_{l_{1}}^{sim}$')
    plt.plot(space, conc_exp, '^', label=r'$E_{C}^{expt}$')
    plt.plot(space, coh_exp, '>', label=r'$C_{l_{1}}^{expt}$')
    plt.plot(t, Ec_teor, label=r'$E_{C}^{the}$')
    plt.plot(t, Coh_teor, label=r'$C_{l_{1}}^{the}$')
    plt.xlabel(r'$p_{00}$')
    #plt.ylabel('Quantumness')
    plt.legend(bbox_to_anchor=(1.2, 1.0), loc='upper right')
    plt.savefig('cohXEc_xstate.pdf', dpi=100, bbox_inches='tight')
    plt.show()

In [ ]:
# Chamar a função para gerar os gráficos
generate_graphs(space, conc_sim, coh_sim, conc_exp, coh_exp, Ec_teor, Coh_teor)

# Teste dos códigos

In [1]:
import numpy as np

In [3]:
def Psi00(th):
    return np.array([[np.cos(th)],[0],[0],[np.sin(th)]]);

def Psi01(ph):
    return np.array([[0],[np.sin(ph)],[np.cos(ph)],[0]]);

def Psi11(ph):
    return np.array([[0],[np.cos(ph)],[-np.sin(ph)],[0]]);

def Psi10(th):
    return np.array([[-np.sin(th)],[0],[0],[np.cos(th)]]);

In [25]:
th=np.pi/4
ph=np.pi/4

def rhoX(th,ph,p00,p01,p10,p11):
    return p00*(Psi00(th)*Psi00(th).T) +  p01*(Psi01(ph)*Psi01(ph).T) + p10*(Psi10(th)*Psi10(th).T) + p11*(Psi11(ph)*Psi11(ph).T)

rhoX(th,ph,1/2,1/2,0,0)

array([[0.25, 0.  , 0.  , 0.25],
       [0.  , 0.25, 0.25, 0.  ],
       [0.  , 0.25, 0.25, 0.  ],
       [0.25, 0.  , 0.  , 0.25]])

In [10]:
import math

In [26]:
def rhoX_(p):
    
    p0 = p/3
    p1 = (1-p)/3
    p2= (2*p)/3
    p3 = (2*(1-p))/3

    tt = math.pi/4
    #rho2 = np.zeros((4,4), dtype=complex)
    rho2 = np.zeros((4,4))
    
    rho2[0][0] = p0*np.power(np.cos(tt),2)+p2*np.power(np.sin(tt),2)
    rho2[0][3] = (p0-p2)*np.cos(tt)*np.sin(tt)
    rho2[3][0] = np.conj(rho2[0][3])
    rho2[1][1] = p1*np.power(np.sin(tt),2)+p3*np.power(np.cos(tt),2)
    rho2[1][2] = (p1-p3)*np.cos(tt)*np.sin(tt)
    rho2[2][1] = np.conj(rho2[1][2])
    rho2[2][2] = p1*np.power(np.cos(tt),2)+p3*np.power(np.sin(tt),2)
    rho2[3][3] = p0*np.power(np.sin(tt),2)+p2*np.power(np.cos(tt),2)
    return rho2

In [31]:
rhoX_(0.01)

array([[ 0.005     ,  0.        ,  0.        , -0.00166667],
       [ 0.        ,  0.495     , -0.165     ,  0.        ],
       [ 0.        , -0.165     ,  0.495     ,  0.        ],
       [-0.00166667,  0.        ,  0.        ,  0.005     ]])